In [1]:
import joblib # to save and load the models
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from collections import defaultdict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

import seaborn as sns

In [2]:
data = pd.read_csv('data/adult.data', names=['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
                                              'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
                                              'hrs_per_week', 'native_country', 'income'])

data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hrs_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
data_test = pd.read_csv('data/adult.test', names=['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
                                              'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
                                              'hrs_per_week', 'native_country', 'income'])
data_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hrs_per_week,native_country,income
0,|1x3 Cross validator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.


In [4]:
#data.info()

In [5]:
#data_test.info()

In [6]:
data_test.dropna(inplace=True)

In [7]:
# There's no nans
#data.isna().sum()

In [8]:
data = pd.concat([data, data_test], ignore_index=True)

In [9]:
data.income.unique()

array([' <=50K', ' >50K', ' <=50K.', ' >50K.'], dtype=object)

In [10]:
data['income'] = data['income'].replace({' <=50K.': ' <=50K', ' >50K.': ' >50K'})
data.income.unique()

array([' <=50K', ' >50K'], dtype=object)

In [11]:
# in this column the order does not matter
data.workclass.unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [12]:
# we must keep the order in this encoding
data.education.unique()

array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
       ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
       ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
       ' Preschool', ' 12th'], dtype=object)

In [13]:
edu = [' Preschool', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' 10th', ' 11th', ' 12th',
       ' HS-grad', ' Prof-school', ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' Bachelors',
       ' Masters',' Doctorate']

In [14]:
data.occupation.unique()

array([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
       ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
       ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
       ' Tech-support', ' ?', ' Protective-serv', ' Armed-Forces',
       ' Priv-house-serv'], dtype=object)

In [15]:
#data.occupation.value_counts()

In [16]:
#sns.countplot(data=data, y='occupation', hue='income')

In [17]:
# auxiliar calculations to decide how to order the occupations columns
aux = data[['occupation', 'income']].copy()
a = pd.DataFrame(aux.groupby(['occupation', 'income']).size())
a = {'clerical': (756/4784), 'af': (4/10), 'craft': (1355/4665), 'manager': (2867/3117), 'farm': (172/1308),
     'clean': (135/1911), 'mach': (365/2605), 'other': (196/4612), 'priv': (3/229), 'prof': (2704/3304),
     'prot_serv': (307/669), 'sales': (1455/3953), 'tech':(411/1009), 'transport': (478/1838) }
# sorted by the ratio of >50k / <= 50k by profession
dict(sorted(a.items(), key=lambda item: item[1]))

{'priv': 0.013100436681222707,
 'other': 0.042497831743278404,
 'clean': 0.0706436420722135,
 'farm': 0.13149847094801223,
 'mach': 0.1401151631477927,
 'clerical': 0.15802675585284282,
 'transport': 0.2600652883569097,
 'craft': 0.2904608788853162,
 'sales': 0.36807487983809767,
 'af': 0.4,
 'tech': 0.40733399405351833,
 'prot_serv': 0.4588938714499253,
 'prof': 0.8184019370460048,
 'manager': 0.919794674366378}

In [18]:
ocupation = [' Priv-house-serv', ' Other-service', ' Handlers-cleaners', ' Farming-fishing', ' Machine-op-inspct',
             ' Adm-clerical', ' Transport-moving', ' Craft-repair', ' Sales', ' Armed-Forces', ' Tech-support',
             ' Protective-serv', ' Prof-specialty', ' Exec-managerial']

In [19]:
data.loc[data.workclass == ' ?']

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hrs_per_week,native_country,income
27,54,?,180211.0,Some-college,10.0,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0.0,0.0,60.0,South,>50K
61,32,?,293936.0,7th-8th,4.0,Married-spouse-absent,?,Not-in-family,White,Male,0.0,0.0,40.0,?,<=50K
69,25,?,200681.0,Some-college,10.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K
77,67,?,212759.0,10th,6.0,Married-civ-spouse,?,Husband,White,Male,0.0,0.0,2.0,United-States,<=50K
106,17,?,304873.0,10th,6.0,Never-married,?,Own-child,White,Female,34095.0,0.0,32.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48682,61,?,265201.0,Some-college,10.0,Married-civ-spouse,?,Husband,White,Male,0.0,0.0,14.0,United-States,<=50K
48769,21,?,212661.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
48800,73,?,144872.0,HS-grad,9.0,Married-civ-spouse,?,Husband,White,Male,0.0,0.0,25.0,Canada,<=50K
48812,81,?,26711.0,Assoc-voc,11.0,Married-civ-spouse,?,Husband,White,Male,2936.0,0.0,20.0,United-States,<=50K


In [20]:
#data.loc[data.native_country == ' ?']

In [21]:
data.replace(' ?', np.NaN, inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True, drop=True)

In [22]:
le = preprocessing.LabelEncoder()
sc = preprocessing.MaxAbsScaler()
oe = preprocessing.OrdinalEncoder(categories=[edu, ocupation])


In [ ]:
def preprocess(df):
    mappings = {}
    y = df.income
    X = df.drop(columns=['income'])
    objList = X.select_dtypes(include="object").columns.to_list()
    y = le.fit_transform(y)
    
    for feat in objList:
        X[feat] = X[feat].astype('category')
        mappings[feat] = (X[feat].cat.categories, X[feat].cat.codes)
        X[feat] = X[feat].cat.codes
        
    #X = pd.DataFrame(oe.transform(X[['workclass', 'education', 'marital_status', 'occupation', 'relationship',
    #                        'race', 'sex', 'native_country']]))
    #print(X.head())
    X = sc.fit_transform(X)
    
    return X, y, mappings

In [ ]:
X, y, mappings = preprocess(data)
#a.head()

In [23]:
def preprocessing_v3(df):
    y = df.income
    X = df.drop(columns=['income'])
    objList = X.select_dtypes(include="object").columns.to_list()
    objList.remove('education')
    y = le.fit_transform(y)
    #oe.fit(X[['education', 'occupation']])
    X[['education', 'occupation']] = pd.DataFrame(oe.fit_transform(X[['education', 'occupation']]))
    X = pd.get_dummies(X, columns=['workclass', 'marital_status', 'relationship',
                                   'race', 'sex', 'native_country'])
    X = sc.fit_transform(X)
    
    return X, y

In [24]:
X_3, y_3 = preprocessing_v3(data)

In [ ]:
def preprocess_v2(df):
    y = df.income
    X = df.drop(columns=['income'])
    
    y = le.fit_transform(y)
    X = pd.get_dummies(X, columns=['workclass', 'education', 'marital_status', 'occupation', 'relationship',
                                   'race', 'sex', 'native_country'])
    X = sc.fit_transform(X)
    
    return X, y

In [ ]:
X_2, y_2 = preprocess_v2(data)

In [ ]:
def train_knn(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    param_grid = [
        {'n_neighbors': [9, 11, 13, 15, 17],
         'weights': ['uniform', 'distance'],
         'leaf_size': [3, 5, 10, 15, 20],
         'p': [1,2] }
    ]

    knn = KNeighborsClassifier()

    gs_knn = GridSearchCV(knn,
                        param_grid=param_grid,
                        scoring='accuracy', n_jobs=-1,
                        cv=10)

    gs_knn.fit(X_train, y_train)
    y_pred = gs_knn.predict(X_test)
    print(gs_knn.best_params_)
    print(gs_knn.score(X_test, y_test))
    print(classification_report(y_test, y_pred))
    
    return gs_knn


In [ ]:
# Using ordinal encoding
train_knn(X, y)

In [ ]:
# Using onehot encoding
train_knn(X_2, y_2)

In [ ]:
knn = train_knn(X_3, y_3)

In [ ]:
# saving the model to disk
filename = 'knn.sav'
joblib.dump(knn, filename)

In [ ]:
# loading the model from disk
loaded_model = joblib.load('knn.sav')
result = loaded_model.score(X_test, y_test)

# Ramdom Forest

In [ ]:
def train_rf(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=0)
    
    param_grid = [
        {'criterion': ['gini','entropy'],
        'max_depth': [10,30,60,None],
        'max_leaf_nodes': [10,30,60,None],
        'max_features': ['auto', 'sqrt'],
        'min_samples_leaf': [1,2,3],
        'min_samples_split': [2,4],
        'cn_estimators': (500,800,1000,1200)}
    ]

    rf = RandomForestClassifier()

    gs_rf = GridSearchCV(rf,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=5)

    gs_rf.fit(X_train, y_train)
    print(gs_rf.best_params_)
    print(gs_rf.best_score_)
    print(gs_rf.score(X_test, y_test))


In [ ]:
# Triyng with standard parameters
# With the second preprocess function
RF_classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
RF_classifier.fit(X_2, y_2) 

In [ ]:
all_accuracies_RF = cross_val_score(estimator=RF_classifier, X=X_2, y=y_2, cv=3)
print('cross_val_score: ', all_accuracies_RF)
print('Mean: ', all_accuracies_RF.mean())
print('Standard deviation: ', all_accuracies_RF.std())

In [ ]:
#y_pred = classifier_best.predict(X_test)

In [ ]:
#print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
#print('accuracy_score: ', accuracy_score(y_test, y_pred))